In [20]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pathlib
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

In [3]:
waste_types = pathlib.Path('/home/loupy/code/Loupy2023/get_it_right/raw_data/waste_types')
garb_cardboard = list(waste_types.glob('cardboard/*'))
garb_glass = list(waste_types.glob('glass/*'))
garb_paper = list(waste_types.glob('paper/*'))
garb_metal = list(waste_types.glob('metal/*'))
garb_plastic = list(waste_types.glob('plastic/*'))
garb_trash = list(waste_types.glob('trash/*'))
img = Image.open(str(garb_cardboard[100]))
img_height, img_width =  img.size
batch_size = 32

In [4]:
img_height

512

In [5]:
img_width
img.size

(512, 384)

In [6]:
data = image_dataset_from_directory(waste_types)
class_names = data.class_names
print(class_names)

Found 2390 files belonging to 5 classes.


2023-03-08 18:11:59.065257: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-08 18:11:59.070867: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-08 18:11:59.071527: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-603AUFB): /proc/driver/nvidia/version does not exist
2023-03-08 18:11:59.182094: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


['cardboard', 'glass', 'metal', 'paper', 'plastic']


# Split


In [7]:
pip install split-folders


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pathlib
dataset_url = pathlib.Path('/home/loupy/code/Loupy2023/get_it_right/raw_data/waste_types')

In [9]:
batch_size = 16
img_height = 512
img_width = 384

In [10]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_url,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2390 files belonging to 5 classes.
Using 1912 files for training.


In [11]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_url,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2390 files belonging to 5 classes.
Using 478 files for validation.


In [12]:
len(train_ds)

120

In [85]:
train_ds
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


2023-03-08 16:46:08.515250: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 37748736 exceeds 10% of free system memory.


(16, 512, 384, 3)
(16,)


# Building model

In [27]:
# define the pre-processing layers
from tensorflow.keras import layers
shape=(img_height, img_width, 3)
shape_def = (batch_size, img_height, img_width, 3)


#preprocessing_layers = tf.keras.Sequential([
    #layers.experimental.preprocessing.Rescaling(1./255),
# Define the model architecture

def initialize_model():
   # model = Sequential([
        #layers.experimental.preprocessing.Rescaling(1./255)])
#         model.add(Conv2D(5, 3, padding= 'same', activation='relu', input_shape=shape)),
#         model.add(MaxPooling2D()),
#         model.add(Conv2D(16, 3,padding= 'same', activation='relu')),
#         model.add(MaxPooling2D()),
#         model.addFlatten()
#         model.add(Dense(5, activation='relu'))
#         model.add(Dense(5, activation='softmax'))
        
    model = Sequential([
      layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
      layers.RandomFlip("horizontal"),
      layers.RandomRotation(0.1),
      #layers.RandomCrop(input_shape[0], input_shape[1]),
      layers.Conv2D(5, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(10, activation='relu'),
      layers.Dense(5)
    ])  
    
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
       # model.build(input_shape=shape_def)
    
# Compile the model
#     model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
    
    return model

In [28]:
model = initialize_model()

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 512, 384, 3)       0         
                                                                 
 random_flip_1 (RandomFlip)  (None, 512, 384, 3)       0         
                                                                 
 random_rotation_1 (RandomRo  (None, 512, 384, 3)      0         
 tation)                                                         
                                                                 
 conv2d_2 (Conv2D)           (None, 512, 384, 5)       140       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 256, 192, 5)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 256, 192, 16)     

In [30]:
#Train the model
epochs=10
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=1
)

2023-03-08 18:31:04.272455: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 37748736 exceeds 10% of free system memory.
2023-03-08 18:31:36.269088: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 37748736 exceeds 10% of free system memory.
2023-03-08 18:31:38.268288: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 37748736 exceeds 10% of free system memory.
2023-03-08 18:31:41.518322: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 37748736 exceeds 10% of free system memory.
2023-03-08 18:31:42.014509: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 62914560 exceeds 10% of free system memory.


120/120 [==============================] - 1188s 9s/step - loss: 1.7953 - accuracy: 0.2317 - val_loss: 1.6058 - val_accuracy: 0.2615


In [ ]:
import cv2
# Load the pre-trained Faster R-CNN model
net = cv2.dnn.readNetFromTensorflow('faster_rcnn.pb')
# Load the input image
image = cv2.imread('image.jpg')
# Create a blob from the input image
blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(300, 300), mean=(0, 0, 0), swapRB=True, crop=False)
# Set the input for the network
net.setInput(blob)
# Run the forward pass to get the output of the network
detections = net.forward()
# Loop over the detections
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        # Get the coordinates of the bounding box
        box = detections[0, 0, i, 3:7] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
        (startX, startY, endX, endY) = box.astype("int")
        # Draw the bounding box and label on the image
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
        label = '{0:.2f}%'.format(confidence * 100)
        cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
# Show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)